In [13]:
import cv2
import sys
import numpy as np
from time import time

In [21]:
# get feed
video_capture = cv2.VideoCapture(0)

# set resolution
video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 960)

# set fps
frame_rate = 30
prev = 0

# set fps counter
time1 = 0

# read image and convert it to proper formats
_ , current_frame = video_capture.read()
gray_first = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)#b2lb l gray scale ashan a 3rf a3ml minus ala level wahed bs msh 3
gaus_first = cv2.GaussianBlur(gray_first,(21,21),0)
blur_first = cv2.blur(gaus_first,(5,5))

# array that will hold the differances of the last 30 frames
diffs = []

while video_capture.isOpened():
    
    blur_last = blur_first
    
    ret , current_frame = video_capture.read()

    # set fps to 30
    time_elapsed = time() - prev
    if time_elapsed > 1./frame_rate:
        prev = time()
        
        # if image not taken, take the last image
        if current_frame is None:
            current_frame = blur_last

        # convert new image
        gray_first = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
        gaus_first = cv2.GaussianBlur(gray_first,(21,21),0)
        blur_first = cv2.blur(gaus_first,(5,5))

        # Find the absolute difference between frames
        diff = cv2.absdiff(blur_first, blur_last)
        # threshold between values bigger and smaller than 20
        thresh = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)[1]
        # store differance between frames
        diff_num = round(np.mean(thresh),2)

        # append to list to find average differance
        if(len(diffs)<60):
            diffs.append(diff_num)
        else:
            diffs.pop(0)
            diffs.append(diff_num)


        cv2.putText(thresh,
                        "diff: {}".format(int(np.mean(diffs))),
                        (10,50),
                       cv2.FONT_HERSHEY_PLAIN, 2,
                       (255), 3)

        cv2.imshow('Video',thresh)
        if cv2.waitKey(1) & 0xFF == ord('x'):#bdos letter x ashan a2fl el camera
            break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()